# TOPIC MODELING

### Dependencies

In [1]:
import pandas as pd
import string
import warnings, re
warnings.simplefilter('ignore')
from itertools import chain

import gensim
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.models.coherencemodel import CoherenceModel 


import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


import pyLDAvis
import pyLDAvis.gensim_models
from wordcloud import WordCloud
import matplotlib.pyplot as plt

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Patil\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Patil\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Patil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
d:\binghamton\semester1\IS2 502\topicmodeling\env\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


### Loading data

In [2]:
data = pd.read_excel('metoo_tweets_dec2017.xlsx')
data.head()

,column_a,text,favorited,favoritecount,replytosn,created,truncated,replytosid,id,replytouid,statussource,screenname,retweetcount,isretweet,retweeted,longitude,latitude,location
0,1,American Harem.. #MeToo https://t.co/HjExLJdGuF,0.0,0,NaN,2017-11-29T23:59:00,0.0,NaN,9.360000e+17,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",ahmediaTV,0.0,0.0,0.0,NaN,NaN,NaN
1,2,@johnconyersjr @alfranken why have you guys ...,0.0,0,johnconyersjr,2017-11-29T23:59:00,0.0,NaN,9.360000e+17,266149840,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",JesusPrepper74,0.0,0.0,0.0,NaN,NaN,NaN
2,3,Watched Megan Kelly ask Joe Keery this A.M. if...,0.0,0,NaN,2017-11-29T23:59:00,1.0,NaN,9.360000e+17,NaN,"<a href=""http://twitter.com/download/android"" ...",DemerisePotvin,0.0,0.0,0.0,NaN,NaN,NaN
3,4,Women have been talking about this crap the en...,0.0,0,NaN,2017-11-29T23:59:00,0.0,NaN,9.360000e+17,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",TheDawnStott,0.0,0.0,0.0,NaN,NaN,NaN
4,5,.@BetteMidler please speak to this sexual assa...,0.0,15,NaN,2017-11-29T23:59:00,0.0,NaN,9.360000e+17,NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",scottygirl2014,11.0,0.0,0.0,NaN,NaN,NaN


In [3]:
filtered_data = data.filter(['text'], axis=1)
filtered_data.head()
filtered_data = filtered_data[filtered_data["text"].apply(lambda x: type(x) == str)]

### Cleaning Data

In [4]:
# Removing the false and nanvalues
stop_words = set(stopwords.words('english'))
stop_words  |= set(['ï¿½ï¿½', 'ï¿½', 'x000d', 'rcc', 'mex000d', 'leaguex000daccording', 'jr', 'donohue', 15, 1, 200, 'x000dx000d', 'im', 'u'])

def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'ï¿½', '', text)
    text = text.lower()
    text = re.sub("#[A-Za-z0-9_]+","", text)
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = strip_non_alphanum(text)
    text =  ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word.lower() not in stop_words])
    return text

filtered_data['clean_text'] = filtered_data['text'].apply(clean_text)
filtered_data['clean_tokens'] = filtered_data['clean_text'].apply(lambda x: x.split())

In [5]:
filtered_data.head()

,text,clean_text,clean_tokens
0,American Harem.. #MeToo https://t.co/HjExLJdGuF,american harem,"[american, harem]"
1,@johnconyersjr @alfranken why have you guys ...,guy resigned yet liberal hypocrisy,"[guy, resigned, yet, liberal, hypocrisy]"
2,Watched Megan Kelly ask Joe Keery this A.M. if...,watched megan kelly ask joe keery rub finger h...,"[watched, megan, kelly, ask, joe, keery, rub, ..."
3,Women have been talking about this crap the en...,woman talking crap entire time finally someone...,"[woman, talking, crap, entire, time, finally, ..."
4,.@BetteMidler please speak to this sexual assa...,please speak sexual assault interview,"[please, speak, sexual, assault, interview]"


In [8]:

# # Instantiate LDA model
LSI = gensim.models.LsiModel

df = filtered_data[0: 100000]
# df = filtered_data


# create dictionary'
dictionary = corpora.Dictionary(df['clean_tokens'])

# Total number of non-zeroes in the BOW matrix (sum of the number of unique words per document over the entire corpus).
# print(dictionary.token2id)


# # Create document term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['clean_tokens'] ]
# print(doc_term_matrix)


number_of_topics = 10
words = 10
LSIModel = LSI(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)
print(LSIModel.print_topics(num_topics=number_of_topics, num_words=words))


d:\binghamton\semester1\IS2 502\topicmodeling\env\lib\site-packages\gensim\models\lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
d:\binghamton\semester1\IS2 502\topicmodeling\env\lib\site-packages\gensim\models\lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
d:\binghamton\semester1\IS2 502\topicmodeling\env\lib\site-packages\gensim\models\lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
d:\binghamton\semester1\IS2 502\topicmodeling\env\lib\site-packages\gensim\models\lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` name

[(0, '0.381*"sexual" + 0.322*"misconduct" + 0.312*"milano" + 0.306*"allegation" + 0.305*"president" + 0.303*"list" + 0.302*"state" + 0.300*"america" + 0.300*"full" + 0.300*"united"'), (1, '0.436*"woman" + 0.433*"movement" + 0.335*"year" + 0.334*"time" + 0.331*"person" + 0.216*"amp" + 0.139*"trump" + 0.135*"silence" + 0.126*"story" + 0.107*"magazine"'), (2, '0.658*"woman" + -0.276*"movement" + -0.265*"time" + -0.222*"person" + -0.218*"year" + 0.153*"amp" + 0.117*"sexual" + -0.098*"magazine" + -0.086*"gave" + -0.085*"interview"'), (3, '0.421*"farrow" + 0.421*"dylan" + 0.395*"woody" + 0.391*"allen" + 0.388*"revolution" + 0.385*"spared" + 0.117*"via" + -0.064*"amp" + -0.034*"person" + -0.034*"year"'), (4, '-0.502*"amp" + 0.296*"woman" + -0.175*"franken" + -0.173*"al" + -0.170*"iâ" + -0.167*"set" + -0.160*"donâ" + -0.160*"stone" + -0.159*"pâ" + -0.159*"leeann"'), (5, '0.296*"year" + 0.283*"person" + -0.238*"like" + -0.232*"asked" + -0.232*"comment" + -0.232*"interview" + -0.231*"deemed" + -